# **get ready**

In [ ]:
!pip install parsivar

!pip install hazm
!pip install transformers
#!pip install torch
!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 48.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.8/238.8 kB 28.5 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9296 sha256=fdd252463812355c0d3d3230640bb60e2eddb3c7fbc585520b8fdd55bac56281
  Stored in directory: /root/.cache/pip/wheels/bc/be/39/c37ad168eb2ff644c9685f52554440372129450f0b8ed203dd
Successfully built flashtext
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resol

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import numpy as np
import pandas as pd
import re
#import torch
import tensorflow as tf
from gensim.models import Word2Vec
from hazm import Normalizer as HazmNormalizer, stopwords_list, word_tokenize, Stemmer
from parsivar import Normalizer as ParsivarNormalizer, Tokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset


# reading data

In [ ]:
dataset_path = '/content/drive/MyDrive/data/train_data.csv'
data = pd.read_csv(dataset_path)

In [ ]:
unique_tags = data['tags'].unique()
print(unique_tags)

['استانها' 'اجتماعی' 'اقتصادی' 'سیاسی' 'فرهنگی' 'بین الملل' 'ورزشی'
 'رسانه ها' 'بازار' 'الشرق الأوسط' 'دانش و محیط زیست' 'جهان' 'ایران'
 'NRS-Import' 'فرهنگ و هنر' 'آلمان' 'دیدگاه' 'دوره\u200cهای زبان آلمانی'
 'ورزش' 'اقتصاد' 'گوناگون' 'شبکه\u200cهای اجتماعی' 'دویچه وله'
 'Community D' 'آلمانی پیش\u200cرفته' 'ویدئو > گزارش' 'استان ها > مرکزی'
 'استان ها > قم' 'علمی و دانشگاهی > صنفی، فرهنگی'
 'اقتصادی > عمران و اشتغال' 'اقتصادی > تولید و تجارت' 'اجتماعی > سلامت'
 'اجتماعی > آموزش و پرورش' 'سیاسی > دولت'
 'فرهنگی و هنری > گردشگری و میراث' 'اقتصادی > اقتصاد کلان'
 'ورزشی > فوتبال، فوتسال' 'سیاسی > سیاست خارجی' 'اجتماعی > حوادث، انتظامی'
 'سیاسی > مجلس' 'اجتماعی > محیط زیست' 'سیاسی > اندیشه امام و رهبری'
 'اجتماعی > خانواده' 'سیاسی > حقوقی و قضایی' 'سیاسی > سیاست داخلی'
 'استان ها > لرستان' 'علمی و دانشگاهی > علم و فناوری ایران'
 'استان ها > گیلان' 'اجتماعی > جامعه، شهری' 'سیاسی > دفاعی - امنيتی'
 'علمی و دانشگاهی > پژوهش' 'گرافیک > اینفوگرافیک' 'استان ها > خوزستان'
 'اقتصادی > انرژی' 'ف

In [ ]:
tags = {
    "اجتماعی": "social",
    "اقتصادی": "economics",
    "استانها": "Iran-states",
    "بین الملل": "international",
    "سیاسی": "politics",
    "ورزشی": "scientific-cultural-sports",
    "جهان": "international",
    "ایران": "Iran-states",
    "اقتصاد ایران": "economics",
    "رسانه ها": "scientific-cultural-sports",
    "بازار": "economics",
    "الشرق الأوسط": "international",
    "دانش و محیط زیست": "scientific-cultural-sports",
    "NRS-Import": "international",
    "فرهنگ و هنر": "scientific-cultural-sports",
    "آلمان": "international",
    "دیدگاه": "politics",
    "دوره‌های زبان آلمانی": "scientific-cultural-sports",
    "ورزش": "scientific-cultural-sports",
    "اقتصاد": "economics",
    "گوناگون": "scientific-cultural-sports",
    "شبکه‌های اجتماعی": "scientific-cultural-sports",
    "دویچه وله": "international",
    "Community D": "scientific-cultural-sports",
    "آلمانی پیش‌رفته": "scientific-cultural-sports",
    "ویدئو > گزارش": "scientific-cultural-sports",
    "استان ها > مرکزی": "Iran-states",
    "استان ها > قم": "Iran-states",
    "علمی و دانشگاهی > صنفی، فرهنگی": "scientific-cultural-sports",
    "اقتصادی > عمران و اشتغال": "economics",
    "اقتصادی > تولید و تجارت": "economics",
    "اجتماعی > سلامت": "social",
    "اجتماعی > آموزش و پرورش": "social",
    "سیاسی > دولت": "politics",
    "فرهنگی و هنری > گردشگری و میراث": "scientific-cultural-sports",
    "اقتصادی > اقتصاد کلان": "economics",
    "ورزشی > فوتبال، فوتسال": "scientific-cultural-sports",
    "سیاسی > سیاست خارجی": "politics",
    "اجتماعی > حوادث، انتظامی": "social",
    "سیاسی > مجلس": "politics",
    "اجتماعی > محیط زیست": "social",
    "سیاسی > اندیشه امام و رهبری": "politics",
    "اجتماعی > خانواده": "social",
    "سیاسی > حقوقی و قضایی": "politics",
    "سیاسی > سیاست داخلی": "politics",
    "استان ها > لرستان": "Iran-states",
    "علمی و دانشگاهی > علم و فناوری ایران": "scientific-cultural-sports",
    "استان ها > گیلان": "Iran-states",
    "اجتماعی > جامعه، شهری": "social",
    "سیاسی > دفاعی - امنيتی": "politics",
    "علمی و دانشگاهی > پژوهش": "scientific-cultural-sports",
    "گرافیک > اینفوگرافیک": "scientific-cultural-sports",
    "استان ها > خوزستان": "Iran-states",
    "اقتصادی > انرژی": "economics",
    "فرهنگی و هنری > سینما و تئاتر": "scientific-cultural-sports",
    "استان ها > سمنان": "Iran-states",
    "استان ها > خراسان رضوی": "Iran-states",
    "سیاسی > انرژی هسته‌ای": "politics",
    "علمی و دانشگاهی > علم و فناوری جهان": "scientific-cultural-sports",
    "استان ها > البرز": "Iran-states",
    "استان ها > تهران": "Iran-states",
    "علمی و دانشگاهی > جهاد دانشگاهی": "scientific-cultural-sports",
    "ورزشی > سایر ورزش‌ها": "scientific-cultural-sports",
    "استان ها > هرمزگان": "Iran-states",
    "اقتصادی > ارتباطات و فناوری اطلاعات": "economics",
    "علمی و دانشگاهی > آموزش": "scientific-cultural-sports",
    "ورزشی > کشتی، رزمی": "scientific-cultural-sports",
    "بین الملل > فرامنطقه ای": "international",
    "بین الملل > آسیا،خاورمیانه": "international",
    "استان ها > اصفهان": "Iran-states",
    "استان ها > چهارمحال و بختیاری": "Iran-states",
    "فرهنگی و هنری > تجسمی و موسیقی": "scientific-cultural-sports",
    "فرهنگی و هنری > دین و اندیشه": "scientific-cultural-sports",
    "فرهنگی و هنری > ادبیات و کتاب": "scientific-cultural-sports",
    "استان ها > بوشهر": "Iran-states",
    "فرهنگی و هنری > فرهنگ حماسه": "scientific-cultural-sports",
    "فرهنگی و هنری > رسانه": "scientific-cultural-sports",
    "استان ها > سیستان و بلوچستان": "Iran-states",
    "ورزشی > توپ و تور": "scientific-cultural-sports",
    "استان ها > کهگیلویه و بویراحمد": "Iran-states",
    "استان ها > کرمانشاه": "Iran-states",
    "استان ها > مازندران": "Iran-states",
    "استان ها > ایلام": "Iran-states",
    "استان ها > زنجان": "Iran-states",
    "ورزشی > جهان ورزش": "scientific-cultural-sports",
    "استان ها > کرمان": "Iran-states",
    "استان ها > یزد": "Iran-states",
    "استان ها > اردبیل": "Iran-states",
    "ورزشی > ورزش بانوان": "scientific-cultural-sports",
    "استان ها > خراسان جنوبی": "Iran-states",
    "بین الملل > فلسطین": "international",
    "ایسنا+ > ایسنا+": "scientific-cultural-sports",
    "استان ها > فارس": "Iran-states",
    "استان ها > گلستان": "Iran-states",
    "عکس > خبری": "scientific-cultural-sports",
    "بایگانی": "scientific-cultural-sports",
    "خبرخوان": "scientific-cultural-sports",
    "دانش و فناوری": "scientific-cultural-sports",
    "فرهنگی": "scientific-cultural-sports"
}

data['tags'] = data['tags'].replace(tags)

data['tags'].fillna('unknown', inplace=True)

print(data['tags'].unique())

['Iran-states' 'social' 'economics' 'politics'
 'scientific-cultural-sports' 'international'
 'سیاسی > انرژی هسته\u200e\u200cای' 'ايران' 'unknown']


In [13]:
import pandas as pd

dataset_path = '/content/drive/MyDrive/data/train_data.csv'

data = pd.read_csv(dataset_path)

tag_counts = data['tags'].value_counts()

for tag, count in tag_counts.items():
    print(f"{tag}: {count}")

بایگانی: 1944
جهان: 1658
ایران: 1188
بین الملل: 972
سیاسی: 921
سیاسی > دولت: 506
اجتماعی: 493
اقتصادی: 478
استانها: 433
سیاسی > سیاست خارجی: 418
اجتماعی > سلامت: 407
آلمان: 310
سیاسی > مجلس: 266
سیاسی > سیاست داخلی: 261
دانش و محیط زیست: 203
سیاسی > حقوقی و قضایی: 166
اقتصاد: 133
فرهنگی: 126
سیاسی > دفاعی - امنيتی: 110
اقتصادی > اقتصاد کلان: 108
اجتماعی > حوادث، انتظامی: 79
فرهنگ و هنر: 75
ورزشی > فوتبال، فوتسال: 68
علمی و دانشگاهی > آموزش: 67
اقتصادی > تولید و تجارت: 63
اجتماعی > جامعه، شهری: 61
اقتصادی > عمران و اشتغال: 49
بین الملل > آسیا،خاورمیانه: 47
اجتماعی > آموزش و پرورش: 46
فرهنگی و هنری > گردشگری و میراث: 42
علمی و دانشگاهی > علم و فناوری ایران: 38
اقتصادی > انرژی: 37
اجتماعی > محیط زیست: 37
سیاسی > انرژی هسته‎‌ای: 35
اجتماعی > خانواده: 35
سیاسی > اندیشه امام و رهبری: 34
ورزش: 33
اقتصاد ایران: 32
دیدگاه: 31
بین الملل > فرامنطقه ای: 24
علمی و دانشگاهی > علم و فناوری جهان: 23
ورزشی: 22
خبرخوان: 20
ايران: 20
استان ها > کهگیلویه و بویراحمد: 18
فرهنگی و هنری > تجسمی و موسیقی: 17
ف

In [ ]:
tags_to_drop = ['سیاسی > انرژی هسته‌ای', 'ايران', 'unknown','سیاسی > انرژی هسته\u200e\u200cای']

data = data[~data['tags'].isin(tags_to_drop)]

print(data['tags'].unique())

['Iran-states' 'social' 'economics' 'politics'
 'scientific-cultural-sports' 'international']


In [ ]:
unique_tags = data['tags'].unique()
unmapped_tags = [tag for tag in unique_tags if tag not in tags.values()]
print(f"Unmapped tags: {unmapped_tags}")

Unmapped tags: []


# add  preprocessing

In [ ]:
X = data['title'] + ' ' + data['description']
y = data['tags']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

hazm_normalizer = HazmNormalizer()
parsivar_normalizer = ParsivarNormalizer()
stopwords = set(stopwords_list())
stemmer = Stemmer()

def preprocess_text(text):
    text = hazm_normalizer.normalize(text)
    text = parsivar_normalizer.normalize(text)
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\s+", " ", text)
    tokens = word_tokenize(text)
    tokens = [stemmer.stem(token) for token in tokens if token not in stopwords]
    return ' '.join(tokens)

X_train = X_train.apply(preprocess_text)
X_test = X_test.apply(preprocess_text)

# logestic regression with tfidf


In [ ]:

tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = tfidf_vectorizer.transform(X_test).toarray()

logreg_model = LogisticRegression(max_iter=1000, C=10, solver='lbfgs')

logreg_model.fit(X_train_tfidf, y_train)

y_pred = logreg_model.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")



Accuracy: 0.777016129032258
F1 Score: 0.7766655599748348


# logestic with word2vec

In [ ]:
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)


In [ ]:
X_train_w2v = np.array([vectorize(sentence) for sentence in X_train])
X_test_w2v = np.array([vectorize(sentence) for sentence in X_test])
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

logreg_model = LogisticRegression(max_iter=1000)

logreg_model.fit(X_train_w2v, y_train)

y_pred = logreg_model.predict(X_test_w2v)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")








# **Best result in logestic regression**

In [ ]:
#X = data['description']
X = data['title'] + ' ' + data['description']

y = data['tags']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

log_reg = LogisticRegression(max_iter=100000)
log_reg.fit(X_train_vectorized, y_train)
y_pred_log_reg = log_reg.predict(X_test_vectorized)

log_reg_accuracy = accuracy_score(y_test, y_pred_log_reg)
log_reg_f1 = f1_score(y_test, y_pred_log_reg, average='weighted')

print(f"Logistic Regression Accuracy: {log_reg_accuracy}")
print(f"Logistic Regression F1 Score: {log_reg_f1}")


Logistic Regression Accuracy: 0.8012096774193549
Logistic Regression F1 Score: 0.8015051049514994


use it for test_data.csv

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

test_data = pd.read_csv('/content/drive/MyDrive/data/test_data.csv')

test_data['text'] = test_data['title'] + ' ' + test_data['description']
test_data['text'] = test_data['text']

test_vectorized = vectorizer.transform(test_data['text'])

test_predictions = log_reg.predict(test_vectorized)

test_labels = label_encoder.inverse_transform(test_predictions)

submission_df = pd.DataFrame(test_labels, columns=['prediction'])

submission_df.to_csv('submission.csv', index=False)


______________


# **low accuracy in MLP and LSTM **

In [ ]:
sentences = [sentence.split() for sentence in X_train]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)

def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train_w2v = np.array([vectorize(sentence) for sentence in X_train])
X_test_w2v = np.array([vectorize(sentence) for sentence in X_test])


from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

model = Sequential()
model.add(Dense(100, input_dim=100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train_w2v, y_train, epochs=10, batch_size=32, validation_split=0.2)

y_pred = model.predict(X_test_w2v)
y_pred_classes = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, LSTM, Dropout, Flatten

X_train_w2v_reshaped = X_train_w2v.reshape(X_train_w2v.shape[0], X_train_w2v.shape[1], 1)
X_test_w2v_reshaped = X_test_w2v.reshape(X_test_w2v.shape[0], X_test_w2v.shape[1], 1)

model_cnn_lstm = Sequential()
model_cnn_lstm.add(Conv1D(64, kernel_size=3, activation='relu', padding='same', input_shape=(100, 1)))
model_cnn_lstm.add(MaxPooling1D(pool_size=2))
model_cnn_lstm.add(Dropout(0.2))
model_cnn_lstm.add(Conv1D(128, kernel_size=3, activation='relu', padding='same'))
model_cnn_lstm.add(MaxPooling1D(pool_size=2))
model_cnn_lstm.add(Dropout(0.2))
model_cnn_lstm.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model_cnn_lstm.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model_cnn_lstm.add(Dense(32, activation='relu'))
model_cnn_lstm.add(Dense(len(label_encoder.classes_), activation='softmax'))

model_cnn_lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history_cnn_lstm = model_cnn_lstm.fit(X_train_w2v_reshaped, y_train, epochs=20, batch_size=32, validation_split=0.2)

y_pred_cnn_lstm = model_cnn_lstm.predict(X_test_w2v_reshaped)
y_pred_classes_cnn_lstm = np.argmax(y_pred_cnn_lstm, axis=1)

accuracy_cnn_lstm = accuracy_score(y_test, y_pred_classes_cnn_lstm)
f1_cnn_lstm = f1_score(y_test, y_pred_classes_cnn_lstm, average='weighted')

print(f"Accuracy CNN-LSTM: {accuracy_cnn_lstm}")
print(f"F1 Score CNN-LSTM: {f1_cnn_lstm}")


# **BONUS PART _ MLP**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, GlobalMaxPooling1D, LSTM
from keras.utils import to_categorical

df = pd.read_csv('/content/drive/MyDrive/data/train_data.csv')

import re

def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.strip().lower()  # Convert to lowercase
    return text

df['title'] = df['title'].apply(clean_text)
df['description'] = df['description'].apply(clean_text)

df['text'] = df['title'] + ' ' + df['description']

label_map = {

    'اجتماعی': 'social',
    'اقتصادی': 'economics',
    'استانها': 'Iran-states',
    'بین‌الملل': 'International',
    'سیاسی': 'politics',
    'علمی فرهنگی ورزشی': 'scientific-cultural-sports'
}

df['label'] = df['tags'].map(label_map)
labels = df['label'].unique().tolist()
label_dict = {label: i for i, label in enumerate(labels)}
df['label'] = df['label'].map(label_dict)

max_words = 10000
max_len = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
word_index = tokenizer.word_index

data = pad_sequences(sequences, maxlen=max_len)
labels = to_categorical(df['label'], num_classes=len(label_dict))

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_len))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_dict), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_split=0.2)

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')


Epoch 1/30
125/125 [==============================] - 4s 22ms/step - loss: 0.7876 - accuracy: 0.8091 - val_loss: 0.6308 - val_accuracy: 0.8073
Epoch 2/30
125/125 [==============================] - 2s 20ms/step - loss: 0.4981 - accuracy: 0.8279 - val_loss: 0.4751 - val_accuracy: 0.8279
Epoch 3/30
125/125 [==============================] - 3s 22ms/step - loss: 0.3616 - accuracy: 0.8657 - val_loss: 0.4189 - val_accuracy: 0.8455
Epoch 4/30
125/125 [==============================] - 5s 38ms/step - loss: 0.2363 - accuracy: 0.9196 - val_loss: 0.3721 - val_accuracy: 0.8751
Epoch 5/30
125/125 [==============================] - 3s 24ms/step - loss: 0.1318 - accuracy: 0.9632 - val_loss: 0.4078 - val_accuracy: 0.8766
Epoch 6/30
125/125 [==============================] - 3s 25ms/step - loss: 0.0658 - accuracy: 0.9851 - val_loss: 0.4665 - val_accuracy: 0.8741
Epoch 7/30
125/125 [==============================] - 3s 24ms/step - loss: 0.0272 - accuracy: 0.9959 - val_loss: 0.5170 - val_accuracy: 0.8751

In [ ]:
import pandas as pd
from keras.preprocessing.sequence import pad_sequences

test_df = pd.read_csv('/content/drive/MyDrive/data/test_data.csv')

test_df['title'] = test_df['title'].apply(clean_text)
test_df['description'] = test_df['description'].apply(clean_text)
test_df['text'] = test_df['title'] + ' ' + test_df['description']

test_sequences = tokenizer.texts_to_sequences(test_df['text'])
test_data = pad_sequences(test_sequences, maxlen=max_len)

predictions = model.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)

reverse_label_map = {v: k for k, v in label_dict.items()}
predicted_labels = [reverse_label_map[c] for c in predicted_classes]

submission_df = pd.DataFrame(predicted_labels, columns=['prediction'])

submission_df.to_csv('submission2.csv', index=False)


32/32 [==============================] - 0s 3ms/step


put number for main class in submission file

In [18]:
import pandas as pd

class_mapping = {
    'social': 0,
    'economics': 1,
    'Iran-states': 2,
    'international': 3,
    'politics': 4,
    'scientific-cultural-sports': 5
}

submission_df = pd.read_csv('/content/submission.csv')

submission_df['prediction'] = submission_df['prediction'].map(class_mapping)

submission_df['prediction'] = submission_df['prediction'].astype(int)

submission_df.to_csv('submission_log_numeric.csv', index=False)